# 訓練 😉

### 載入函式庫

In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split 
from sklearn import tree
from sklearn import svm
from sklearn.metrics import accuracy_score
from collections import defaultdict
import pandas as pd
import data_generator

### 讀入資料

In [2]:
df = pd.read_csv("data/student_data_30000.csv")
df.head()

,Avg sleep time,Avg study time,Avg video game time,BMI,In a relationship,Family financial status,GPA,Laptop brand,TOEIC grade,IQ,Grade
0,7,3,13,33.0,Yes,high,2.6,Apple,700,142,80+
1,7,3,12,24.0,No,average,2.6,Leveno,910,115,60-
2,8,3,11,31.0,Yes,high,3.2,Acer,545,128,60-
3,6,4,8,32.0,No,low,4.2,Asus,455,115,80+
4,7,4,10,32.0,Yes,average,2.9,Asus,945,148,60-


### 特徵

In [3]:
features = df.drop("Grade", axis=1)
features = pd.get_dummies(features, drop_first=True)
features.head()

,Avg sleep time,Avg study time,Avg video game time,BMI,GPA,TOEIC grade,IQ,In a relationship_Yes,Family financial status_high,Family financial status_low,Laptop brand_Apple,Laptop brand_Asus,Laptop brand_Dell,Laptop brand_HP,Laptop brand_Leveno
0,7,3,13,33.0,2.6,700,142,1,1,0,1,0,0,0,0
1,7,3,12,24.0,2.6,910,115,0,0,0,0,0,0,0,1
2,8,3,11,31.0,3.2,545,128,1,1,0,0,0,0,0,0
3,6,4,8,32.0,4.2,455,115,0,0,1,0,1,0,0,0
4,7,4,10,32.0,2.9,945,148,1,0,0,0,1,0,0,0


### 類別

In [4]:
label = df["Grade"]
label.head()

0    80+
1    60-
2    60-
3    80+
4    60-
Name: Grade, dtype: object

### 將資料分成訓練集(0.8)與測試集(0.2)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.20)

### Decision Tree 🌲

In [6]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
pred_label = clf.predict(X_test)

In [8]:
accuracy_score(y_test, pred_label)

0.7843333333333333

### Support Vector Machine 🗿

In [9]:
clf = svm.SVC(gamma='scale')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [10]:
pred_label = clf.predict(X_test)

In [11]:
accuracy_score(y_test, pred_label)

0.5663333333333334

In [12]:
# !jupyter nbconvert --to script training.ipynb